# Anotações e Entendimento Geral do Paper Pix2Pix GAN:
A arquitetura da rede Pix2Pix original proposta é um modelo de redes adversárias generativas (GANs) voltado para tarefas de tradução de imagem para imagem. Ela consiste em dois componentes principais: o Gerador (Generator) e o Discriminador (Discriminator).

### Gerador (Generator):
O gerador utiliza uma arquitetura de encoder-decoder com skip connections, semelhante à U-Net.
 - Encoder: c64-c128-c256-c512-c512-c512-c512-c512
 - Decoder: cd512-cd512-cd512-c512-c256-c128-c64

### Discriminador (Discriminator):
O discriminador utiliza uma arquitetura chamada PatchGAN. Em vez de classificar a imagem inteira como real ou falsa, ele divide a imagem em patches menores e faz a classificação em cada um desses patches.
- Arquitetura: c64-c128-c256-c512
- Camada Final: aplica uma convolução que transforma o mapa de ativação em uma dimensão (1D), seguida por uma função sigmoid para produzir a probabilidade de autenticidade.


## Desenvolvimento do Código:

### Importando bibliotecas

In [ ]:
from numpy import ones
from numpy import zeros
from numpy.random import randint

from keras.optimizers import Adam
from keras.initializers import RandomNormal

from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Conv2DTranspose
from keras.layers import BatchNormalization

from matplotlib import pyplot as plt
from tensorflow.keras.utils import plot_model


### Carregando os Dados e Criando Datasets

Inicialmente a ideia eh utilizar o mesmo dataset do paper e depois testar em algum dos outros datases propostos para esse trabalho

In [ ]:
# nao deu certo ainda :(

In [2]:
# fazendo o modelo em si

def Descriminador(img_shape):
    init = RandomNormal(stddev=0.02) # como feito no paper

    in_src_img = Input(shape=img_shape)
    in_target_img = Input(shape=img_shape)

    merged_img = Concatenate()[in_src_img, in_target_img]

    # arquitetura da rede em si:
    layer = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged_img)
    layer = LeakyReLU(alpha=0.2)(layer)
    # C128
    layer = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer)
    layer = BatchNormalization()(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    # C256
    layer = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer)
    layer = BatchNormalization()(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    # C512
    layer = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer)
    layer = BatchNormalization()(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    # second last output layer
    layer = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(layer)
    layer = BatchNormalization()(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    # patch output
    layer = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(layer)
    patch_out = Activation('sigmoid')(layer)

    # define model
	model = Model([in_src_img, in_target_img], patch_out)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    
	return model

TabError: inconsistent use of tabs and spaces in indentation (<string>, line 34)